In [2]:
import pandas as pd

train_df = pd.read_csv('data/train.csv')
train_df.head()

,id,target
0,gL5fWrgNPU.jpg,0
1,KKWXJDN8uM.png,1
2,Wb2I0CXlAY.jpg,0
3,G7PLhKpaf7.jpeg,1
4,DEUkeefz6Z.jpg,0


In [1]:
from PIL import Image
with Image.open("data/images/gL5fWrgNPU.jpg") as img:
    img.load()

    img = img.resize((32, 32))
    img.save('new_gL5fWrgNPU.jpg')
img.size

(32, 32)

In [114]:
import pathlib
import tensorflow as tf
# import tensorflow_datasets as tfds

batch_size = 32
img_height = 32
img_width = 32


# train_ds = tf.keras.utils.image_dataset_from_directory(
#     pathlib.Path('data/images'),
#     validation_split = 0.2,
#     subset='training',
#     seed=123,
#     image_size=(img_height, img_width),
#     batch_size=batch_size
# )

1. Соединить соответстующие изображения и их метки
2. Попробовать обучить дискриминатор, основанный на CNN и получить какие-нибудь результаты

In [37]:
from PIL import Image
import numpy as np
with Image.open("data/images/gL5fWrgNPU.jpg") as img:
    img.load()

    img = img.resize((32, 32))
    img_np = np.array(img) / 255.0
    # img_np = img_np / 255.0
    # img.save('new_gL5fWrgNPU.jpg')
# img.size
img_np[0][0]

array([0.45098039, 0.37647059, 0.31764706])

In [35]:
from keras.preprocessing import image

X_test=image.load_img('data/images/gL5fWrgNPU.jpg',target_size=(32,32)); #loading image and then convert it into grayscale and with it's target size 
X_test=image.img_to_array(X_test); #convert image into array
X_test[0][0]

array([69., 53., 80.], dtype=float32)

In [19]:
import pandas as pd

X_data = {'name_file': None, 'val_arr': None, 'label': None}
train_df = pd.read_csv('data/train.csv')
sp_train_df = train_df
# sp_train_df['val_arr'] = sp_train_df

sp_train_df.head()

,id,target
0,gL5fWrgNPU.jpg,0
1,KKWXJDN8uM.png,1
2,Wb2I0CXlAY.jpg,0
3,G7PLhKpaf7.jpeg,1
4,DEUkeefz6Z.jpg,0


Создание датафрейма с значениями RGB

In [4]:
from pathlib import Path
import pandas as pd
from PIL import Image

import tensorflow as tf
from tensorflow.keras import layers, models

import numpy as np

p = Path('data/images')
list_name_img = []
list_var_arr = []
for x in p.glob("*"):
    x_rprefix = str(x).removeprefix('data\images\\')
    x_rprefix = x_rprefix[:10]
    list_name_img.append(x_rprefix)

    img = Image.open(x)
    img_resize = img.resize((32, 32))
    list_var_arr.append(np.array(img_resize) / 255.0)

list_name_img_df = pd.DataFrame({'id': list_name_img, 'val_arr': list_var_arr})
list_name_img_df.head()

,id,val_arr
0,01a7MiLURN,"[[[0.5882352941176471, 0.36470588235294116, 0...."
1,01SPjdJuM4,"[[[0.6431372549019608, 0.803921568627451, 0.95..."
2,05q5J7VWr4,"[[[0.6862745098039216, 0.7294117647058823, 0.7..."
3,0666kmCAAC,"[[[0.3764705882352941, 0.3686274509803922, 0.3..."
4,07aHL0dVg4,"[[[0.43529411764705883, 0.35294117647058826, 0..."


Датафрейм с тренировочными данными

In [5]:
train_df = pd.read_csv('data/train.csv')
train_df_id = train_df['id'].str[:10]
train_df_target = train_df['target']
train_df_new = pd.concat([train_df_id, train_df_target], axis=1)
train_df_new

,id,target
0,gL5fWrgNPU,0
1,KKWXJDN8uM,1
2,Wb2I0CXlAY,0
3,G7PLhKpaf7,1
4,DEUkeefz6Z,0
...,...,...
1007,fqoVycdxLV,1
1008,TN2ZOiNI5e,0
1009,2WMWt6CAWQ,0
1010,fQCY5n2vcF,0


Объединение двух датафреймов

In [6]:
result_df = pd.merge(train_df_new, list_name_img_df, on='id', how='left')
result_df

,id,target,val_arr
0,gL5fWrgNPU,0,"[[[0.45098039215686275, 0.3764705882352941, 0...."
1,KKWXJDN8uM,1,"[[[0.9254901960784314, 0.9254901960784314, 0.9..."
2,Wb2I0CXlAY,0,"[[[0.13333333333333333, 0.11372549019607843, 0..."
3,G7PLhKpaf7,1,"[[[0.10588235294117647, 0.09411764705882353, 0..."
4,DEUkeefz6Z,0,"[[[0.3333333333333333, 0.39215686274509803, 0...."
...,...,...,...
1007,fqoVycdxLV,1,"[[[0.8588235294117647, 0.9333333333333333, 0.9..."
1008,TN2ZOiNI5e,0,"[[[0.2901960784313726, 0.15294117647058825, 0...."
1009,2WMWt6CAWQ,0,"[[[0.01568627450980392, 0.0196078431372549, 0...."
1010,fQCY5n2vcF,0,"[[[0.24705882352941178, 0.2235294117647059, 0...."


1. Преобразовать тренировочный датасет значения id
2. Правильно соединить два датафрейма
3. Обучить

Обучение примитивной CNN-сети

In [11]:
(train_images, train_labels) = np.array(result_df['val_arr'])[:800], np.array(result_df['target'])[:800]

In [13]:
(test_images, test_labels) = np.array(result_df['val_arr'])[800:], np.array(result_df['target'])[800:]

In [15]:
train_images.astype('float32')
# tf_train_images = tf.keras.ops.convert_to_tensor(train_images.astype('float32'))

ValueError: setting an array element with a sequence.

In [126]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.summary()

c:\mArt\dataScience\kaggle\attempt_at_writing\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 4, 4, 64)       │        36,928 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 56,320 (220.00 KB)

 Trainable params: 56,320 (220.00 KB)

 Non-trainable params: 0 (0.00 B)

In [127]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2))

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 4, 4, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │        65,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 122,050 (476.76 KB)

 Trainable params: 122,050 (476.76 KB)

 Non-trainable params: 0 (0.00 B)

In [133]:
model.compile(optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', 'binary_crossentropy'])

history = model.fit(train_images, train_labels.astype('float32'), epochs=5,
    validation_data=(test_images, test_labels.astype('float32')))

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print(test_acc)

ValueError: setting an array element with a sequence.